<a href="https://colab.research.google.com/github/Gaeseobok/gsb-ml/blob/main/realtimeEmoRecogOpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

딥페이스를 사용한 표정 탐지

In [ ]:
!pip install deepface

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/

In [ ]:
# realtime video demo for face emotion recognition

import cv2
from deepface import DeepFace
import matplotlib.pyplot as plt
import os

from google.colab import files
from google.colab.patches import cv2_imshow

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

'''
cap = cv2.VideoCapture(1)
# check if the webcam is open correctly
if not cap.isOpened():
  cap = cv2.VideoCapture(0)
if not cap.isOpened():
  raise IOError("Cannot open webcam")
'''

filePath = 'Facial Expression.mp4'

if os.path.isfile(filePath):	# 해당 파일이 있는지 확인
    # 영상 객체(파일) 가져오기
    cap = cv2.VideoCapture(filePath)
    
    # 프레임을 정수형으로 형 변환
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))	# 영상의 넓이(가로) 프레임
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))	# 영상의 높이(세로) 프레임
    
    frame_size = (frameWidth, frameHeight)
    print('frame_size={}'.format(frame_size))

    # 정수 형태로 변환하기 위해 round
    round_w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    round_h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) # 카메라에 따라 값이 정상적, 비정상적

    # fourcc 값 받아오기, *는 문자를 풀어쓰는 방식, *'DIVX' == 'D', 'I', 'V', 'X'
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')

    # 웹캠으로 찰영한 영상을 저장하기
    # cv2.VideoWriter 객체 생성, 기존에 받아온 속성값 입력
    writer = cv2.VideoWriter('Facial Expression output.mp4', fourcc, fps, (round_w, round_h), isColor = True)

    while True:
      retval,frame = cap.read()  # read one image from a video

      # 한 장의 이미지(frame)를 가져오기
      # 영상 : 이미지(프레임)의 연속
      # 정상적으로 읽어왔는지 -> retval
      # 읽어온 프레임 -> frame

      if not(retval):	# 프레임정보를 정상적으로 읽지 못하면
          break  # while문을 빠져나가기

      # plt.imshow('frame', frame)	# 프레임 보여주기
      # key = cv2.waitKey(fps)  # fps msec동안 한 프레임을 보여준다
      try:
        result = DeepFace.analyze(frame, actions = ['emotion']) # only detect actions confined to emotions

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # print(faceCascade.empty())
        faces = faceCascade.detectMultiScale(gray,1.1,4)

        # draw a rectangle around the faces
        for(x,y,w,h) in faces:
          cv2.rectangle(frame, (x,y),(x+w, y+h), (0,225,0), 2)

        font = cv2.FONT_HERSHEY_SIMPLEX

        # use putText() method for
        # inserting text on video
        cv2.putText(frame, result['dominant_emotion'],
                (50,50), font, 3, (0,0,255), 2, cv2.LINE_4)
        
        #edge_color = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
        
        writer.write(frame) # 영상 데이터만 저장. 소리는 X
        cv2_imshow(frame)
        #plt.imshow(edge_color)

        print(result['dominant_emotion'])
      
      except:
        print("No face detected")

      if cv2.waitKey(2) & 0xFF == ord('q'):
        break

    # result.to_video('result.mp4')
    # files.download('result.mp4')

    cap.release()
    writer.release()
    cv2.destroyAllWindows()
else:
    print("파일이 존재하지 않습니다.")  